- Update config.yaml
- Update params.yaml
- Update the entity
- Update the configuration manager in src config
- Update the components
- Update the pipeline
- Update the main.py
- Update the dvc.yaml

In [1]:
import os
os.getcwd()

'c:\\gitrepository\\Projects-AIML\\CVProject-Chest-Disease-Classification\\experiments'

In [2]:
os.chdir("../")
os.getcwd()

'c:\\gitrepository\\Projects-AIML\\CVProject-Chest-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class PrepareModelConfig:
    """
    This class encapsulates the variables configuration
    """
    root_directory: Path
    model_path: Path
    updated_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [5]:

from src.cnn_classifier.logger import logging

In [6]:
"""
This is the configuration manager for all the stages of pipelines for this Projects.
It will have all the configuration methods encapsulated in a class
"""
from pathlib import Path
from src.cnn_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnn_classifier.utils.common import read_yaml, create_directories
from src.cnn_classifier.entity.config_entity import PrepareModelArchitectureConfig

class ConfigurationManager:
    """
    This is the configuration manager for all the stages of pipelines for this Projects
    """

    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories(directory_list=[self.config.artifacts_root])
    
    def get_model_config(self) -> PrepareModelArchitectureConfig:
        """
        This method is to fetch the config details for Pretrained Models
        """
        config = self.config.prepare_model
        create_directories([config.root_directory])

        prepare_model_config = PrepareModelArchitectureConfig(
            root_directory = Path(config.root_directory)
            ,model_path = Path(config.model_path)
            ,updated_model_path = Path(config.updated_model_path)
            ,params_image_size = self.params.IMAGE_SIZE
            ,params_learning_rate = self.params.LEARNING_RATE
            ,params_include_top = self.params.INCLUDE_TOP
            ,params_weights = self.params.WEIGHTS
            ,params_classes = self.params.CLASSES
        )

        return prepare_model_config

In [7]:
from pathlib import Path
import tensorflow as tf
from src.cnn_classifier.entity.config_entity import  PrepareModelArchitectureConfig

In [8]:
class PrepareModelArchitecture:
    """
    This Class encapsulates the methods for preparing the base models from VGG!6
    """
    def __init__(self, config: PrepareModelArchitectureConfig):
        self.config = config
        self.model: tf.keras.Model = None
        self.updated_model: tf.keras.Model = None

    def get_base_model(self):
        """
        This fetches the pre-trained model from Keras and saves it in our directory
        """
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model: tf.keras.models,
                            classes: int,
                            freeze_all: bool,
                            freeze_till: int,
                            learning_rate: float) -> tf.keras.Model:
        """
        This is the private method that freezes the CNN layers and trains 
        the top most layers of Pretrained models
        """
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_model(self):
        """
        This method calls the private method for preparing the updated model
        """
        self.updated_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_model_path, model=self.updated_model)

        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """ This method is used to save the model - pre-trained model or finetuned model"""
        model.save(path)


In [9]:
STAGE_NAME = "Prepare model architecture"


class PrepareModelArchitecturePipeline:
    """This class and its methods triggers the PrepareModel pipeline to setup model"""
    def __init__(self):
        pass

    def main(self):
        """
        This method triggers all the functions from PrepareModel class
        """
        config = ConfigurationManager()
        prepare_model_config = config.get_model_config()
        prepare_model = PrepareModelArchitecture(config=prepare_model_config)
        prepare_model.get_base_model()
        prepare_model.update_model()

In [10]:
STAGE_NAME = "Prepare model architecture"

try:
    logging.info("*******************")
    logging.info(">>>>>> Started %s <<<<<<", STAGE_NAME)
    obj = PrepareModelArchitecturePipeline()
    obj.main()
    logging.info(">>>>>> %s completed <<<<<<\n\nx==========x", STAGE_NAME)
except Exception as e:
    logging.exception(e)
    raise e

[2024-05-23 10:43:02,345: INFO: 610368863: *******************]
[2024-05-23 10:43:02,346: INFO: 610368863: >>>>>> Started Prepare model architecture <<<<<<]
[2024-05-23 10:43:02,351: INFO: common: YAML file - config\config.yaml loaded successfully]
[2024-05-23 10:43:02,356: INFO: common: YAML file - params.yaml loaded successfully]
[2024-05-23 10:43:02,358: INFO: common: created directory at: artifacts]
[2024-05-23 10:43:02,360: INFO: common: created directory at: artifacts/prepare_model]
[2024-05-23 10:43:02,975: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)    